In [1]:
#based on https://www.kaggle.com/concealberti/covid-week1-weather

In [11]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/lucas_botella/Downloads/tfm-covid-forecast-lb-278ff92b9d29.json"

In [3]:
# Set your own project id here
PROJECT_ID = 'poetic-bison-304616'
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [6]:
table1_stations = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.stations"
)
dataframe_stations = client.list_rows(
    table1_stations,
    selected_fields=[
        bigquery.SchemaField("usaf", "STRING"), #station number, world metherorological org
        bigquery.SchemaField("wban", "STRING"), #wban number, weather bureau army
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("lat", "FLOAT"),
        bigquery.SchemaField("lon", "FLOAT"),
    ],
).to_dataframe()

dataframe_stations

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,usaf,wban,country,lat,lon
0,007018,99999,None,0.000,0.000
1,007026,99999,AF,0.000,0.000
2,007070,99999,AF,0.000,0.000
3,008268,99999,AF,32.950,65.567
4,008307,99999,AF,0.000,0.000
...,...,...,...,...,...
29556,A00029,63820,US,36.380,-88.985
29557,A06854,00115,US,34.264,-116.854
29558,A51256,00451,US,36.699,-93.402
29559,720377,13917,US,35.633,-91.167


In [4]:
table1_gsod2020 = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.gsod2020"
)

dataframe_gsod2020= client.list_rows(table1_gsod2020,
    selected_fields=[
        bigquery.SchemaField("stn", "STRING"), #station number
        bigquery.SchemaField("wban", "STRING"), #station number
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("mo", "INTEGER"),
        bigquery.SchemaField("da", "INTEGER"),
        bigquery.SchemaField("temp", "FLOAT"), #mean temp of the day
        bigquery.SchemaField("dewp", "FLOAT"), #mean_dew_point
        bigquery.SchemaField("slp", "FLOAT"), #mean_sealevel_pressure
        bigquery.SchemaField("wdsp", "FLOAT"), #mean_wind_speed
        bigquery.SchemaField("prcp", "FLOAT"), #total_precipitation
        bigquery.SchemaField("sndp", "FLOAT"), #snow_depth
    ],).to_dataframe()

dataframe_gsod2020

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,stn,wban,year,mo,da,temp,dewp,slp,wdsp,prcp,sndp
0,071200,99999,2020,12,11,46.7,9999.9,991.5,10.1,0.45,999.9
1,702220,26501,2020,1,7,-44.6,9999.9,1026.3,0.1,0.00,999.9
2,714850,99999,2020,10,16,49.7,9999.9,1020.6,24.5,1.69,999.9
3,712110,99999,2020,4,25,54.5,9999.9,1012.3,11.5,0.20,999.9
4,725287,04724,2020,9,3,69.6,9999.9,1007.3,8.4,0.00,999.9
...,...,...,...,...,...,...,...,...,...,...,...
4107416,994220,99999,2020,5,14,76.5,61.9,1019.3,11.1,0.00,999.9
4107417,994220,99999,2020,10,2,76.9,66.6,1014.2,7.6,0.00,999.9
4107418,994220,99999,2020,11,11,76.7,76.7,1006.9,18.7,0.00,999.9
4107419,997993,99999,2020,2,8,24.4,15.7,1008.7,14.5,0.54,999.9


In [80]:
train = pd.read_csv( '/Users/lucas_botella/Downloads/results/training_data_with_weather_info_week_1.csv')
train=train.rename(columns={"Lat": "lat", "Long": "lon"})
train

,Id,Province/State,Country/Region,lat,lon,Date,ConfirmedCases,Fatalities,day_from_jan_first,temp,min,max,stp,slp,dewp,rh,ah,wdsp,prcp,fog
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,22,42.6,33.6,54.9,999.9,1024.3,27.4,0.545709,0.186448,9.4,0.00,0
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0,23,42.0,32.7,55.9,999.9,1020.8,22.8,0.461259,0.163225,14.9,99.99,1
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0.0,0.0,24,40.1,36.9,43.2,999.9,1018.6,34.5,0.801794,0.325375,10.4,0.17,1
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0.0,0.0,25,46.0,37.9,56.3,999.9,1018.0,37.8,0.728175,0.214562,6.1,0.57,1
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0.0,0.0,26,42.8,36.1,53.1,999.9,1014.8,33.2,0.685513,0.231656,10.8,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17887,26378,NaN,Zambia,-15.4167,28.2833,2020-03-20,2.0,0.0,80,70.6,62.6,81.9,999.9,NaN,62.8,0.761545,0.198068,6.0,0.00,0
17888,26379,NaN,Zambia,-15.4167,28.2833,2020-03-21,2.0,0.0,81,71.3,66.2,81.5,999.9,NaN,65.3,0.812047,0.212487,7.1,99.99,1
17889,26380,NaN,Zambia,-15.4167,28.2833,2020-03-22,3.0,0.0,82,72.1,67.1,80.4,999.9,NaN,66.7,0.829815,0.218712,5.0,99.99,1
17890,26381,NaN,Zambia,-15.4167,28.2833,2020-03-23,3.0,0.0,83,71.7,66.2,80.6,999.9,NaN,62.8,0.733343,0.192580,4.2,0.00,0


In [7]:
stations_df= dataframe_stations
twenty_twenty_df= dataframe_gsod2020

In [23]:
stations_df['STN'] = stations_df['usaf'] + '-' + stations_df['wban']
twenty_twenty_df['STN'] = twenty_twenty_df['stn'] + '-' + twenty_twenty_df['wban']
cols_1= list(twenty_twenty_df.columns)
cols_2= list(stations_df.columns)
weather_df = twenty_twenty_df[cols_1].join(stations_df[cols_2].set_index('STN'), on='STN',  how='left', lsuffix='_left', rsuffix='_right')

weather_df['temp'] = weather_df['temp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['slp'] = weather_df['slp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['dewp'] = weather_df['dewp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['wdsp'] = weather_df['wdsp'].apply(lambda x: np.nan if x==999.9 else x)
weather_df['prcp'] = weather_df['prcp'].apply(lambda x: np.nan if x==999.9 else x)
weather_df['sndp'] = weather_df['sndp'].apply(lambda x: np.nan if x==999.9 else x)

# convert everything into celsius
weather_df['temp'] = (weather_df['temp'] - 32) / 1.8
weather_df['dewp'] = (weather_df['dewp'] - 32) / 1.8
    
# compute relative humidity as ratio between actual vapour pressure (computed from dewpoint temperature)
# and saturation vapour pressure (computed from temperature) (the constant 6.1121 cancels out)
weather_df['rh'] = (np.exp((18.678*dewp)/(257.14+dewp))/np.exp((18.678*temp)/(257.14+temp)))

# calculate actual vapour pressure (in pascals)
# then use it to compute absolute humidity from the gas law of vapour 
# (ah = mass / volume = pressure / (constant * temperature))
weather_df['ah'] = ((np.exp((18.678*dewp)/(257.14+dewp))) * 6.1121 * 100) / (461.5 * temp)


weather_df['month']= weather_df['mo']
weather_df['day']= weather_df['da']
weather_df['Date']=pd.to_datetime(weather_df[['year','month','day']])
weather_df['Date2']= weather_df['Date']
weather_df['Date2']= weather_df['Date2'].astype('str')
mo2 = weather_df['Date2'].apply(lambda x: x[5:7])
da2 = weather_df['Date2'].apply(lambda x: x[8:10])
weather_df['day_from_jan_first'] = (da2.apply(int)
                               + 31*(mo2=='02') 
                               + 60*(mo2=='03')
                               + 91*(mo2=='04')  
                              )

In [89]:
weather_df

,stn,wban_left,year,mo,da,temp,dewp,slp,wdsp,prcp,...,country,lat,lon,rh,ah,month,day,Date,Date2,day_from_jan_first
0,071200,99999,2020,12,11,8.166667,NaN,991.5,10.1,0.45,...,FR,48,-2,NaN,NaN,12,11,2020-12-11,2020-12-11,11
1,702220,26501,2020,1,7,-42.555556,NaN,1026.3,0.1,0.00,...,US,64,-156,NaN,NaN,1,7,2020-01-07,2020-01-07,7
2,714850,99999,2020,10,16,9.833333,NaN,1020.6,24.5,1.69,...,CA,50,-127,NaN,NaN,10,16,2020-10-16,2020-10-16,16
3,712110,99999,2020,4,25,12.500000,NaN,1012.3,11.5,0.20,...,CA,49,-123,NaN,NaN,4,25,2020-04-25,2020-04-25,116
4,725287,04724,2020,9,3,20.888889,NaN,1007.3,8.4,0.00,...,US,43,-78,NaN,NaN,9,3,2020-09-03,2020-09-03,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093916,994220,99999,2020,5,14,24.722222,16.611111,1019.3,11.1,0.00,...,US,27,-82,0.603578,0.166398,5,14,2020-05-14,2020-05-14,14
4093917,994220,99999,2020,10,2,24.944444,19.222222,1014.2,7.6,0.00,...,US,27,-82,0.702901,0.194650,10,2,2020-10-02,2020-10-02,2
4093918,994220,99999,2020,11,11,24.833333,24.833333,1006.9,18.7,0.00,...,US,27,-82,1.000000,0.276302,11,11,2020-11-11,2020-11-11,11
4093919,997993,99999,2020,2,8,-4.222222,-9.055556,1008.7,14.5,0.54,...,US,43,-70,0.690754,-0.158629,2,8,2020-02-08,2020-02-08,39


In [93]:
weather_df= weather_df.dropna(subset = ['lat', 'lon'])
weather_df = weather_df.reset_index(drop=True)
train= train.dropna(subset = ['lat', 'lon'])
train = train.reset_index(drop=True)
weather_df.lon= weather_df.lon.astype(int)
weather_df.lat= weather_df.lat.astype(int)
train.lon= train.lon.astype(int)
train.lat= train.lat.astype(int)

CovidWeather=train.merge(weather_df, on=['lat', 'lon', 'day_from_jan_first'], how='left')
CovidWeather['temp_x'] = (CovidWeather['temp_x'] - 32) / 1.8
CovidWeather['dewp_x'] = (CovidWeather['temp_x'] - 32) / 1.8
CovidWeather

,Id,Province/State,Country/Region,lat,lon,Date_x,ConfirmedCases,Fatalities,day_from_jan_first,temp_x,...,STN,usaf,wban_right,country,rh_y,ah_y,month,day,Date_y,Date2
0,1,NaN,Afghanistan,33,65,2020-01-22,0.0,0.0,22,5.888889,...,409420-99999,409420,99999,AF,0.210889,0.081562,10.0,22.0,2020-10-22,2020-10-22
1,2,NaN,Afghanistan,33,65,2020-01-23,0.0,0.0,23,5.555556,...,409420-99999,409420,99999,AF,0.440518,0.301783,10.0,23.0,2020-10-23,2020-10-23
2,2,NaN,Afghanistan,33,65,2020-01-23,0.0,0.0,23,5.555556,...,409420-99999,409420,99999,AF,0.757148,-0.036035,12.0,23.0,2020-12-23,2020-12-23
3,3,NaN,Afghanistan,33,65,2020-01-24,0.0,0.0,24,4.500000,...,409420-99999,409420,99999,AF,0.691457,-0.080483,12.0,24.0,2020-12-24,2020-12-24
4,3,NaN,Afghanistan,33,65,2020-01-24,0.0,0.0,24,4.500000,...,409420-99999,409420,99999,AF,0.367221,0.401653,10.0,24.0,2020-10-24,2020-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106105,26378,NaN,Zambia,-15,28,2020-03-20,2.0,0.0,80,21.444444,...,676650-99999,676650,99999,ZA,0.761545,0.198068,3.0,20.0,2020-03-20,2020-03-20
106106,26379,NaN,Zambia,-15,28,2020-03-21,2.0,0.0,81,21.833333,...,676650-99999,676650,99999,ZA,0.812047,0.212487,3.0,21.0,2020-03-21,2020-03-21
106107,26380,NaN,Zambia,-15,28,2020-03-22,3.0,0.0,82,22.277778,...,676650-99999,676650,99999,ZA,0.829815,0.218712,3.0,22.0,2020-03-22,2020-03-22
106108,26381,NaN,Zambia,-15,28,2020-03-23,3.0,0.0,83,22.055556,...,676650-99999,676650,99999,ZA,0.733343,0.192580,3.0,23.0,2020-03-23,2020-03-23


In [119]:
CovidWeather[(CovidWeather['Country/Region']=='United Kingdom') & (CovidWeather['Date_y'].dt.month==1)].temp_x.mean

<bound method Series.mean of 104087    22.722222
104099    22.388889
104102    24.388889
104116    25.166667
104124    25.777778
            ...    
105327     8.222222
105336     9.888889
105340     9.888889
105341     9.888889
105355     9.888889
Name: temp_x, Length: 110, dtype: float64>

In [37]:
weather_df_copy['Date'].dt.month

0          12
1           1
2          10
3           4
4           9
           ..
4093916     5
4093917    10
4093918    11
4093919     2
4093920     5
Name: Date, Length: 4093921, dtype: int64